Next on the docket:
* find a nice way of limiting planes (Maybe setting nan's on both x y and z? Maybe doing some fancy projection and only plotting the really necessary points?)
* Find a way to limit the amount of planes to be calculated
* Write the Reciprocal control function into the Lattice control function for less waste of source code.


In [1]:
from lattices import *
% matplotlib notebook
np.set_printoptions(threshold=np.nan)

In [6]:
def Reciprocal(a1=d[0], a2=d[1], a3=d[2], indices=np.array([1, 1, 1]),
               colors=d[4], sizes=d[5],
               min_=d[8], max_=d[9], lattice_name=None, verbose=False):
    if lattice_name is not None:
        lattice, basis, lattice_type = chooser(lattice_name, verbose=verbose)
        a1, a2, a3 = lattice
        # Classify the lattice
    else:
        lattice_type = classifier(a1, a2, a3, basis)

        # Rotate the lattice
        a1, a2, a3, basis = rotator(a1, a2, a3, basis,
                                             lattice_type, verbose=verbose)
    
    basis = np.array([0,0,0])
    
    length_basis = np.shape(basis)
    if len(length_basis) == 1:
        n_basis = 1
    elif len(length_basis) > 1:
        n_basis = length_basis[0]

    # Make a list, n_basis long, for the colors and sizes,
    # if they're not specified.
    c_name = colors.__class__.__name__
    if c_name == "str":
        c = colors
        colors = []
        for i in range(n_basis):
            colors.append(c)
    elif c_name == "list" and len(colors) < n_basis:
        c = colors[0]
        colors = []
        for i in range(n_basis):
            colors.append(c)

    s_name = sizes.__class__.__name__
    if s_name == "int" or s_name == "float":
        s = sizes
        sizes = []
        for i in range(n_basis):
            sizes.append(s)
    elif s_name == "list" and len(sizes) < n_basis:
        s = sizes[0]
        sizes = []
        for i in range(n_basis):
            sizes.append(s)
    
    # Next we find the limits:
    lim_type = 'proper'
    r_min, r_max, n_min, n_max = find_limits(lim_type, a1, a2, a3,
                                                      min_, max_)
    
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = generator(a1, a2, a3, basis, colors, sizes,
                                            lim_type, n_min, n_max, r_min,
                                            r_max)
    # Then we get the planes
    h,k,l = indices
    d, planes = reciprocal(a1, a2, a3, h, k, l, r_min, r_max)
    
    
    # Objects to limit to the plot-box
    objects = [atomic_positions, lattice_coefficients, atomic_colors,
               atomic_sizes, lattice_position]
    objects = limiter(atomic_positions, objects, r_min, r_max)
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = objects
    
    # Prune each of the planes
    planes = [(p[0], p[1], plane_limiter(p, r_min, r_max, 1.1)) for p in planes]
    planes = [p for p in planes if not np.isnan(p[2]).all()]
    
    # Create the figure
    fig = plt.figure()
    ax = fig.gca(projection="3d")

    # Plot atoms. For now a single size and color
    ax.scatter(atomic_positions[:, 0], atomic_positions[:, 1],
               atomic_positions[:, 2], c=atomic_colors, s=atomic_sizes)

    # plot the displacement vector
    ax.quiver(0, 0, 0, d[0], d[1], d[2])
    ax.text(d[0] / 2, d[1] / 2, d[2] / 2, '$d$')

    for p in planes:
        ax.plot_surface(p[0], p[1], p[2], color='xkcd:cement', shade=False, alpha = 0.4)
    # Set limits, orthographic projection (so we get the beautiful hexagons),
    # no automatic gridlines, and no axes
    ax.set_aspect('equal')
    ax.set_proj_type('ortho')
    ax.set_xlim3d([r_min[0], r_max[0]])
    ax.set_ylim3d([r_min[1], r_max[1]])
    ax.set_zlim3d([r_min[2], r_max[2]])
    ax.grid(False)
    plt.axis('equal')
    plt.axis('off')

    # make the panes transparent (the plot box)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    plt.show()

#Reciprocal(lattice_name = "simple cubic", indices = (1,1,0))

In [7]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
blargh = ('r', 'r','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,2,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "simple cubic"

#Lattice(lattice_name = LatticeType, colors = blargh, sizes = sizes, max_ = Maxs, verbose=True)
